In [ ]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"xinkaichen97","key":"a31b1965f229c3f21b0282be2d9cdd3f"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config path -p /content

In [ ]:
!pip install ipdb
import os, zipfile, tarfile, ipdb
os.environ['KAGGLE_USERNAME'] = "xinkaichen97" 
os.environ['KAGGLE_KEY'] = "a31b1965f229c3f21b0282be2d9cdd3f"

In [ ]:
!kaggle competitions download -c 11-785-fall-20-homework-3

In [ ]:
import zipfile
with zipfile.ZipFile("train.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("train_labels.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("dev.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("dev_labels.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("test.npy.zip","r") as z:
  z.extractall(".")

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

In [ ]:
!pip install torch_edit_distance

In [ ]:
!pip install python-Levenshtein

In [ ]:
import numpy as np
import Levenshtein
# import torch_edit_distance
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils.rnn as rnn
from torch.nn import CTCLoss
from torch.utils.data import Dataset, DataLoader
from phoneme_list import PHONEME_LIST, PHONEME_MAP
from ctcdecode import CTCBeamDecoder

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
train_features = np.load('train.npy', allow_pickle=True)
dev_features = np.load('dev.npy', allow_pickle=True)
test_features = np.load('test.npy', allow_pickle=True)

In [ ]:
train_labels = np.load('train_labels.npy', allow_pickle=True)
dev_labels = np.load('dev_labels.npy', allow_pickle=True)

In [ ]:
print(train_features[1].shape)

In [ ]:
print(train_labels[0].shape, train_labels[100])

In [ ]:
print(len(PHONEME_LIST), PHONEME_LIST)
print(len(PHONEME_MAP), PHONEME_MAP)

In [ ]:
class SpeechDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
    
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, idx):
        X = self.inputs[idx]
        Y = self.labels[idx] + 1
        return torch.from_numpy(X), torch.from_numpy(Y)

In [ ]:
def collate_fn(data):
  feature_lst = [item[0] for item in data]
  label_lst = [item[1] for item in data]
  features = rnn.pad_sequence(feature_lst)
  labels = rnn.pad_sequence(label_lst, batch_first=True)
  feature_lengths = torch.tensor([sample.shape[0] for sample in feature_lst])
  label_lengths = torch.tensor([sample.shape[0] for sample in label_lst])
  return features.float(), labels.long(), feature_lengths.long(), label_lengths.long()

In [ ]:
train_dataset = SpeechDataset(train_features, train_labels)
dev_dataset = SpeechDataset(dev_features, dev_labels)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [ ]:
print(len(dev_dataset))

In [ ]:
# baseline
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size=13, hidden_size=256, num_layers=3, bidirectional=True)
        self.fc = nn.Linear(512, 42)

    def forward(self, x):
        y = self.lstm(x)[0]
        y, lens = rnn.pad_packed_sequence(y)
        y = self.fc(y).log_softmax(2)
        # y = F.log_softmax(y)
        return y, lens

In [ ]:
# final model
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv1d(in_channels=13, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.lstm = nn.LSTM(input_size=32, hidden_size=512, num_layers=4, bidirectional=True)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 42)

    def forward(self, x, lengths):
        x = x.permute(1, 2, 0)
        x = self.conv(x)
        x = x.permute(2, 0, 1)
        packed_x = rnn.pack_padded_sequence(x, lengths, enforce_sorted=False)
        y = self.lstm(packed_x)[0]
        y, lens = rnn.pad_packed_sequence(y)
        y = self.fc1(y)
        y = self.fc2(y).log_softmax(2)
        return y, lens

In [ ]:
# bigger model (didn't reach A cutoff)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=13, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.lstm = nn.LSTM(input_size=32, hidden_size=512, num_layers=5, bidirectional=True)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 42)

    def forward(self, x, lengths):
        x = x.permute(1, 2, 0)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.permute(2, 0, 1)
        packed_x = rnn.pack_padded_sequence(x, lengths, enforce_sorted=False)
        y = self.lstm(packed_x)[0]
        y, lens = rnn.pad_packed_sequence(y)
        y = self.fc1(y)
        y = self.fc2(y).log_softmax(2)
        return y, lens

In [ ]:
model = Net().cuda()
criterion = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-6)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [ ]:
# validation
def validate(model, val_loader):
    total_distance = 0
    count = 0
    for i, (batch_data, batch_labels, feature_lengths, _) in enumerate(val_loader):
        batch_data = batch_data.cuda()
        feature_lengths = feature_lengths.cuda()
        with torch.no_grad():
          outputs, output_lens = model(batch_data, feature_lengths)
        decoder = CTCBeamDecoder(labels=PHONEME_MAP, beam_width=10, log_probs_input=True)
        beam_results, beam_scores, timesteps, out_lens = decoder.decode(outputs.transpose(0, 1).cpu(), output_lens.cpu())
        # for each example in the batch
        for j in range(batch_data.shape[1]):
          out_labels = beam_results[j][0][:out_lens[j][0]]
          pred = "".join([PHONEME_MAP[n] for n in out_labels])
          actual = "".join([PHONEME_MAP[n] for n in batch_labels[j]])
          distance = Levenshtein.distance(pred, actual)
          total_distance += distance
          count += 1
        torch.cuda.empty_cache()
        del batch_data
        del batch_labels
        del feature_lengths
    assert count == 2332
    avg_distance = total_distance / count
    return avg_distance

In [ ]:
# training
val_distances = []
num_epochs = 2
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i, (batch_data, batch_labels, feature_lengths, label_lengths) in enumerate(train_loader):
        batch_data = batch_data.cuda()
        # batch_labels = batch_labels.cuda()
        feature_lengths = feature_lengths.cuda()
        # label_lengths = label_lengths.cuda()        
        optimizer.zero_grad()
        outputs, out_lens = model(batch_data, feature_lengths)
        loss = criterion(outputs, batch_labels.cuda(), out_lens, label_lengths.cuda())
        loss.backward()
        optimizer.step()
        running_loss = 0.0
        running_loss += loss.item()
        epoch_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.10f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        torch.cuda.empty_cache()
        del batch_data
        del batch_labels
        del feature_lengths
        del label_lengths
        del loss
    print(epoch_loss / 343)
    scheduler.step()
    # if epoch % 5 == 4:
    # val_dist = validate(model, dev_loader)
    # print(val_dist)
    # val_distances.append(val_dist)
print('Finished Training')

[1,   100] loss: 0.0004416916
[1,   200] loss: 0.0005748497
[1,   300] loss: 0.0005436538
0.0524900309285339
[2,   100] loss: 0.0003537426
[2,   200] loss: 0.0003872909
[2,   300] loss: 0.0004654157
0.039893587296367036
Finished Training


In [ ]:
torch.save(model.state_dict(), "model_e25.pt")

In [ ]:
model.load_state_dict(torch.load('model_bigger_e25.pt'))

In [ ]:
class TestDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs
    
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, idx):
        X = self.inputs[idx]
        return torch.from_numpy(X)

In [ ]:
def collate_fn_test(data):
  feature_lst = [item for item in data]
  features = rnn.pad_sequence(feature_lst)
  feature_lengths = torch.tensor([sample.shape[0] for sample in feature_lst])
  return features.float(), feature_lengths.long()

In [ ]:
test_dataset = TestDataset(test_features)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn_test)

In [ ]:
# prediction
def predict(model, test_loader):
    preds = []
    for i, (batch_data, feature_lengths) in enumerate(test_loader):
        batch_data = batch_data.cuda()
        feature_lengths = feature_lengths.cuda()   
        with torch.no_grad():
          outputs, output_lens = model(batch_data, feature_lengths)
        decoder = CTCBeamDecoder(labels=PHONEME_MAP, beam_width=100, log_probs_input=True)
        beam_results, beam_scores, timesteps, out_lens = decoder.decode(outputs.transpose(0, 1).cpu(), seq_lens=output_lens.cpu())
        for j in range(batch_data.shape[1]):
          out_labels = beam_results[j][0][:out_lens[j][0]]
          pred = "".join([PHONEME_MAP[n] for n in out_labels])
          preds.append(pred)
        torch.cuda.empty_cache()
        del batch_data
        del feature_lengths
    return preds

In [ ]:
preds = predict(model, test_loader)

In [ ]:
print(len(preds), preds[2000])

2251 .ykhmnwthDIhDrkeRhktrhstikHWic.memrIimhjizmhstHAf.inoRdrthDhkwntfraRnalijhvDhpAst.


In [ ]:
# generate submission.csv
with open('submission.csv', 'w') as f:
  f.write('id,label\n')
  for i in range(len(preds)):
    f.write(str(i) + ',' + preds[i]+"\n")

In [ ]:
!kaggle competitions submit -c 11-785-fall-20-homework-3 -f submission.csv -m ""

100% 174k/174k [00:02<00:00, 59.8kB/s]
Successfully submitted to 11-785-Fall-20-Homework 3 